<a href="https://colab.research.google.com/github/DoItSon/playdata/blob/main/Python02/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 범주형에 대한 가능한 인코딩 
    - 원핫 인코딩
    - 카운트 인코딩
    - 순서형 인코딩
- 모델은 다음의 모델만 사용
    - LogisticRegression
- 스케일링은 minmax 만 사용
    - MinMaxScaler
- 모든 랜덤 요소에 seed 값은 42
    - 검증셋 사이즈는 0.2

In [452]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split # 검증셋 사이즈 0.2
from sklearn.linear_model import LogisticRegression # 모델
from sklearn.preprocessing import MinMaxScaler # 이걸로만 스케일링 할것
from sklearn.preprocessing import OneHotEncoder 

In [453]:
DATA_PATH = "/content/drive/MyDrive/data/"
SEED = 42

In [454]:
df = pd.read_csv(f"{DATA_PATH}titanic.csv")

df.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 머신러닝 실행

In [371]:
df["gender"] = df["gender"].map({"male":1,"female":0})

In [372]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  1309 non-null   int64  
 1   survived     1309 non-null   int64  
 2   pclass       1309 non-null   int64  
 3   name         1309 non-null   object 
 4   gender       1309 non-null   int64  
 5   age          1046 non-null   float64
 6   sibsp        1309 non-null   int64  
 7   parch        1309 non-null   int64  
 8   ticket       1309 non-null   object 
 9   fare         1308 non-null   float64
 10  cabin        295 non-null    object 
 11  embarked     1307 non-null   object 
dtypes: float64(2), int64(6), object(4)
memory usage: 122.8+ KB


In [373]:
df.isnull().sum()

passengerid       0
survived          0
pclass            0
name              0
gender            0
age             263
sibsp             0
parch             0
ticket            0
fare              1
cabin          1014
embarked          2
dtype: int64

In [374]:
target_train = df

In [375]:
target_train["age"] = target_train["age"].fillna(target_train["age"].mean())
target_train["fare"] = target_train["fare"].fillna(target_train["fare"].mean())
target_train["embarked"] = target_train["embarked"].fillna(target_train["embarked"].mode()[0])
target_train["cabin"] = target_train["cabin"].fillna(target_train["cabin"].mode()[0])
target_train.isnull().sum().sum()

0

In [376]:
target_train.select_dtypes("object").nunique()

name        1307
ticket       929
cabin        186
embarked       3
dtype: int64

In [377]:
enc = OneHotEncoder()
enc.fit_transform(target_train[cols]).toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [378]:
tmp = pd.DataFrame(
    enc.fit_transform(target_train[cols]).toarray(), # 넘파이 배열
    columns = enc.get_feature_names_out()
)
tmp.head()

,"name_Abbing, Mr. Anthony","name_Abbott, Master. Eugene Joseph","name_Abbott, Mr. Rossmore Edward","name_Abbott, Mrs. Stanton (Rosa Hunt)","name_Abelseth, Miss. Karen Marie","name_Abelseth, Mr. Olaus Jorgensen","name_Abelson, Mr. Samuel","name_Abelson, Mrs. Samuel (Hannah Wizosky)","name_Abrahamsson, Mr. Abraham August Johannes","name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)",...,cabin_F G73,cabin_F2,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,embarked_C,embarked_Q,embarked_S
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [379]:
data = pd.concat([target_train,tmp],axis = 1).drop(columns=cols)
data.head()

,passengerid,survived,pclass,gender,age,sibsp,parch,fare,"name_Abbing, Mr. Anthony","name_Abbott, Master. Eugene Joseph",...,cabin_F G73,cabin_F2,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,embarked_C,embarked_Q,embarked_S
0,1,0,3,1,22.0,1,0,7.2500,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,1,0,38.0,1,0,71.2833,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,1,3,0,26.0,0,0,7.9250,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,1,0,35.0,1,0,53.1000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,3,1,35.0,0,0,8.0500,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [380]:
x_data = data.iloc[:,2:] # 내점일수부터 필요하기 때문에 슬라이싱
y_data = data["survived"]

x_data.shape, y_data.shape

((1309, 2431), (1309,))

In [381]:
scaler = MinMaxScaler()

x_data = scaler.fit_transform(x_data)
x_data

array([[1.        , 1.        , 0.27345609, ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.473882  , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.32356257, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 1.        , 0.48014531, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.3721801 , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.3721801 , ..., 1.        , 0.        ,
        0.        ]])

In [382]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1047, 2431), (262, 2431), (1047,), (262,))

In [383]:
model = LogisticRegression(random_state=SEED)

model.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [384]:
pred = model.predict(x_valid)
accuracy_score(y_valid,pred)

0.8549618320610687

# 피쳐 추가하기!

In [455]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'gender', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [456]:
z = df.groupby("embarked")["fare"].agg("std").reset_index().rename(columns={"fare":"항구별fare"})
z

,embarked,항구별fare
0,C,84.185996
1,Q,13.616133
2,S,37.096402


In [457]:
x = df.groupby("pclass")["fare"].agg("mean").reset_index().rename(columns={"fare":"pclass별fare"})
x

,pclass,pclass별fare
0,1,87.508992
1,2,21.179196
2,3,13.302889


In [458]:
features = df.merge(z,how="left",on="embarked") 
features.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,항구별fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,37.096402
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.185996
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,37.096402
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,37.096402
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,37.096402


In [443]:
features = features.merge(x,how="left",on="pclass")
features.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,항구별fare,pclass별fare
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,C23 C25 C27,S,37.117346,13.331087
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,84.185996,87.508992
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,C23 C25 C27,S,37.117346,13.331087
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,37.117346,87.508992
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,C23 C25 C27,S,37.117346,13.331087


In [459]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  1309 non-null   int64  
 1   survived     1309 non-null   int64  
 2   pclass       1309 non-null   int64  
 3   name         1309 non-null   object 
 4   gender       1309 non-null   object 
 5   age          1046 non-null   float64
 6   sibsp        1309 non-null   int64  
 7   parch        1309 non-null   int64  
 8   ticket       1309 non-null   object 
 9   fare         1308 non-null   float64
 10  cabin        295 non-null    object 
 11  embarked     1307 non-null   object 
 12  항구별fare      1307 non-null   float64
dtypes: float64(3), int64(5), object(5)
memory usage: 143.2+ KB


In [460]:
features.isnull().sum()

passengerid       0
survived          0
pclass            0
name              0
gender            0
age             263
sibsp             0
parch             0
ticket            0
fare              1
cabin          1014
embarked          2
항구별fare           2
dtype: int64

In [463]:
target_train = features

In [464]:
target_train["age"] = target_train["age"].fillna(target_train["age"].mean())
target_train["fare"] = target_train["fare"].fillna(target_train["fare"].mean())
target_train["embarked"] = target_train["embarked"].fillna(target_train["embarked"].mode()[0])
target_train["cabin"] = target_train["cabin"].fillna(target_train["cabin"].mode()[0])
target_train["항구별fare"] = target_train["항구별fare"].fillna(target_train["항구별fare"].mean())

target_train.isnull().sum().sum()

0

In [465]:
x_data = data.iloc[:,2:]
y_data = data["survived"]

x_data.shape, y_data.shape

((1309, 2431), (1309,))

In [466]:
scaler = MinMaxScaler()

x_data = scaler.fit_transform(x_data)
x_data

array([[1.        , 1.        , 0.27345609, ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.473882  , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.32356257, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 1.        , 0.48014531, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.3721801 , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.3721801 , ..., 1.        , 0.        ,
        0.        ]])

In [467]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1047, 2431), (262, 2431), (1047,), (262,))

In [468]:
model = LogisticRegression(random_state=SEED)

model.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [469]:
pred = model.predict(x_valid)
accuracy_score(y_valid,pred)

0.8549618320610687